In [15]:
import pandas as pd 
from keras.models import load_model
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
df_columns = pd.read_csv('data/12-23대기오염nan처리.csv')

In [3]:
df_columns.columns

Index(['측정일시', '이산화질소', '오존', '일산화탄소', '아황산', '미세', '초미세'], dtype='object')

In [103]:
df_columns['측정일시'] = pd.to_datetime(df_columns['측정일시'], format='%Y%m%d')

In [104]:
df = df_columns.set_index('측정일시')

In [105]:
df.tail()

,이산화질소,오존,일산화탄소,아황산,미세,초미세
측정일시,,,,,,
2023-12-07,0.026,0.0184,0.50,0.0033,62.2,16.5
2023-12-08,0.029,0.0300,0.56,0.0031,84.2,22.8
2023-12-09,0.027,0.0379,0.61,0.0029,72.3,30.7
2023-12-10,0.025,0.0209,0.67,0.0029,80.2,35.6
2023-12-11,0.015,0.0227,0.40,0.0024,7.2,3.0


In [123]:
model_path = '../model/SO2_model.h5'  # 파일 경로 수정
model = load_model(model_path)

In [132]:
from joblib import dump, load

# 데이터셋 준비 (이미 원-핫 인코딩 및 NaN 처리 완료)
data = df.values

# 데이터 정규화
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# 데이터셋 분할
n_train = int(0.9 * len(data_scaled))
train_data = data_scaled[:n_train]
test_data = data_scaled[n_train:]


In [133]:

# 최신 데이터 시퀀스 추출
seq_len = 100

In [134]:
# 최신 데이터 준비
latest_data = test_data[-seq_len:].reshape(1, seq_len, -1)


# 예측 수행
next_day_prediction = model.predict(latest_data)


# 예측 결과 역변환
next_day_prediction_transformed = scaler.inverse_transform(np.concatenate((next_day_prediction, np.zeros((next_day_prediction.shape[0], test_data.shape[1]-1))), axis=1))[:,0]

# 예측된 타겟 값 출력
predicted_target_value = next_day_prediction_transformed[0]
print("하루 뒤의 예측된 타겟 값:", predicted_target_value)

1/1 [==============================] - 0s 34ms/step
하루 뒤의 예측된 타겟 값: 0.010253999054431915


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [21]:
df.tail(3)

,이산화질소,오존,일산화탄소,아황산,미세,초미세
측정일시,,,,,,
20231209,0.027,0.0379,0.61,0.0029,72.3,30.7
20231210,0.025,0.0209,0.67,0.0029,80.2,35.6
20231211,0.015,0.0227,0.40,0.0024,7.2,3.0
